In [1]:
#Libraries imported
import pandas as pd 
import os.path 
from datetime import date

In [2]:
#Base Class 
class Get_File: 
#Attributes of class 
    def __init__(self,Filename,Reason_For_Upload,Payment_Period): 
        self.Filename = Filename
        self.Reason_For_Upload = Reason_For_Upload
        self.Payment_Period= Payment_Period
#Check file exists by searching for filename using os.path module   
    def Check(self):
        if os.path.isfile(self.Filename):
            print("file does exist: " + self.Filename) 
            return True
        else:
            print("File does not exist: " + self.Filename)     
            return False
#Uploads excel file as dataframe. Print statement requires datetime module    
    def Upload(self):
        Get_Date=date.today()
        GD=Get_Date.strftime("%d/%m/%y")
        print("File Uploaded on",GD,"for",self.Payment_Period,"for",self.Reason_For_Upload)
        return pd.read_excel(self.Filename)
#Saves amended dataframe to excel    
    def Save_File (self,df, Saved_Name):
        Get_Date=date.today()
        GD=Get_Date.strftime("%d/%m/%y")
        df.to_excel(Saved_Name,index=False)                                         
        print(self.Saved_Name,"successfully saved on",GD)
        return 


In [3]:
#1st Derived class: inherits attributes & functions from Get_File 
class Amend_File(Get_File): 
    def __init__(self,Filename,Reason_For_Upload,Payment_Period,Saved_Name):
        super(Amend_File,self).__init__(Filename,Reason_For_Upload,Payment_Period)
        self.Saved_Name = Saved_Name 
#These columns are redundant         
    def Delete_Columns(self, df): 
        print("Uneccessary Columns deleted")
        return df.drop(['Org Ref ', 'Org ','Payment Method ', 'Payment Reference', 
        'Date FX  Rate Applied ','FX Contract No.','Fee','Payment Status','Currency Org Pays '], axis=1)   
        print("FX Column renamed")    
#Report only included FX Rate going one way so another FX column will be added 
    def Rename_Column (self, df):
        return df.rename(columns={'FX Rate': 'FX Rate from Foreign Currency to GBP'})
#New column created showing FX Rate going from GBP to foreign currency     
    def Add_Column (self,df):
        df['FX Rate from GBP to Foreign Currency']=df.apply (lambda row: row['Currency Amount '] / row['Amount Paid By Org ']
        , axis = 1)
        print("New FX Column added")
        return df
#Without doing this dates print as Datetime y/m/d/h/ in excel 
    def Change_Issued_Date(self,df): 
        df['Date Payment Issued '] = df['Date Payment Issued ' ].dt.strftime ('%d/%m/%y')
        print ("Issued Date format changed")
        return df
    def Change_Received_Date(self,df):
        df['Date Payment Received '] = df['Date Payment Received ' ].dt.strftime ('%d/%m/%y')
        print("Received Date format changed")
        return df 

In [ ]:
#2nd Derived class: Inheriting from both Get_File & Amend_File
class Member_Single_Month_Request(Amend_File): 
    def __init__(self,Filename,Reason_For_Upload,Payment_Period,Saved_Name,Member_Ref ):
        super(Member_Single_Month_Request,self).__init__(Filename,Reason_For_Upload,Payment_Period,Saved_Name)
#Added Attribute        
        self.Member_Ref=Member_Ref 
#Functions will execute within the Extract_Member Function    
        if super(Member_Single_Month_Request, self).Check(): 
            self.Upload=super(Member_Single_Month_Request, self).Upload() 
            self.Upload=super(Member_Single_Month_Request, self).Add_Column(self.Upload)
            self.Upload=super(Member_Single_Month_Request, self).Change_Received_Date(self.Upload)
#Extracting all infomation needed from file 
    def Extract_Member(self):     
# Its very easy to get member ref wrong so did not want error meeage showing to user here  
     try: 
            df = self.Upload 
            df = df[df['Member Ref '] == self.Member_Ref]
            df = df[['Member Ref ', 'Member Name ','Currency', 'Currency Amount ',
                     'FX Rate from GBP to Foreign Currency', 'Date Payment Received ']]
            print('Dataframe created')
            return df
     
     except: 
            df=None  